In [23]:
from sshtunnel import SSHTunnelForwarder
import paramiko
import sshtunnel
import pymongo
import pprint
from pymongo import MongoClient

In [21]:
def mongo_connect():
    # Try to connect to MongoDB,  exit if not successful.
    try:
        conn = MongoClient('localhost', 27016)
        # print "Connected successfully to MongoDB, instance created!"

    except pymongo.errors.ConnectionFailure, e:
        print "Could not connect to MongoDB: %s" % e

    name = 'test'

    # create database on instance
    if name in conn.database_names():
        # print "Found SA_Voiting_Data Database"
        # print "Connecting to db..."
        db = conn[name]  # Create the database

        # create one collection, called conversations
        conversations = db.conversations_collection
        raw = db.raw_collection
        # print ("Connected!")
    else:
        # print "Creating SA_Voting_Data Database"
        db = conn[name]  # create the database
        conversations = db.conversations_collection
        raw = db.raw_collection

    # collection is a collection in the mongodb instance created above
    # db is the database on the mongodb instance
    # conn is the mongodb instance
    return conn, db, conversations, raw

In [19]:
def clean_conversations_and_add_poi(conversations, poi):
    clean_convos = []
    for conv in conversations:
        clean_conversation = {}
        keys = conv.keys()

        for key in keys:
            if key == "from_addr" or key == "to_addr":
                clean_conversation[key] = retain_msisdn(conv[key])
            elif key == "_id":
                pass
            else:
                clean_conversation[key] = conv[key]

        clean_conversation['poi'] = poi
        clean_convos.append(clean_conversation)
    return clean_convos

def retain_msisdn(msisdn):
    if msisdn is None:
        return u"None"
    
    if msisdn == u"unknown":
        return u""

    # msisdn a regular phone number
    elif msisdn[0:3] == u"+27" and len(msisdn) == 12:
        return msisdn

    # msisdn a mxit phone number
    elif msisdn[0] == u'm':
        return msisdn

    elif msisdn[0:2] == u'07' or msisdn[0:2] == u'27' and len(msisdn) == 10:
        return u'27'+msisdn[1:]

    elif msisdn[0:2] == u'27' and len(msisdn) == 11:
        return msisdn

    # some other type of number we don't know about

    elif msisdn[0:2] != '27' and msisdn[0:2] != '07' and  msisdn[0:3] != '+27':
        return msisdn

In [ ]:
MONGO_HOST = "206.12.98.21"
MONGO_DB = "SA_Voting_Data"
MONGO_USER = "rsampana"
SSH_PORT = 22
MONGO_PORT = 27016

with SSHTunnelForwarder(
    (MONGO_HOST),
    ssh_username=MONGO_USER,
    ssh_pkey="/.ssh/id_rsa",
    remote_bind_address=('127.0.0.1', MONGO_PORT)
) as server:
    client = pymongo.MongoClient('127.0.0.1', server.local_bind_port)
    db = client[MONGO_DB]
    raw = db.get_collection('raw_collection')
    pprint.pprint (raw.find({}).count())
    addresses = list(raw.distinct('from_addr'))
    print len(addresses)

5924547
262172


In [20]:
for address in addresses:
    with SSHTunnelForwarder(
        (MONGO_HOST),
        ssh_username=MONGO_USER,
        ssh_pkey="/.ssh/id_rsa",
        remote_bind_address=('127.0.0.1', MONGO_PORT)
    ) as server:
            client = pymongo.MongoClient('127.0.0.1', server.local_bind_port)
            db = client[MONGO_DB]
            raw = db.get_collection('raw_collection')
            conversation = db.conversations_collection
            convos = raw.find({"$or": [{"from_addr": address}, {"to_addr": address}]}).sort('timestamp', 1)
            cc = list(convos)

            idz = conversation.insert(clean_conversations_and_add_poi(cc, poi=address))
            print "finished with {}".format(address)


finished with +27748024688
finished with +27764914783
finished with +27797247289
finished with +27797513653
finished with +27818660691
finished with +27761924314
finished with +27769544257
finished with +27767555006
finished with +27824721156


2017-07-05 15:43:09,189| ERROR   | Exception: Error reading SSH protocol banner[Errno 54] Connection reset by peer
2017-07-05 15:43:09,193| ERROR   | Traceback (most recent call last):
2017-07-05 15:43:09,194| ERROR   |   File "/Users/RyanSampana/anaconda2/lib/python2.7/site-packages/paramiko/transport.py", line 1749, in run
2017-07-05 15:43:09,195| ERROR   |     self._check_banner()
2017-07-05 15:43:09,196| ERROR   |   File "/Users/RyanSampana/anaconda2/lib/python2.7/site-packages/paramiko/transport.py", line 1897, in _check_banner
2017-07-05 15:43:09,198| ERROR   |     raise SSHException('Error reading SSH protocol banner' + str(e))
2017-07-05 15:43:09,199| ERROR   | SSHException: Error reading SSH protocol banner[Errno 54] Connection reset by peer
2017-07-05 15:43:09,200| ERROR   | 
2017-07-05 15:43:09,217| ERROR   | Could not connect to gateway 206.12.98.21:22 : Error reading SSH protocol banner[Errno 54] Connection reset by peer


BaseSSHTunnelForwarderError: Could not establish session to SSH gateway

In [27]:
def multi(address):
    conn, db, conversations, raw = mongo_connect()
    try:
        convos = raw.find({"$or": [{"from_addr": address}, {"to_addr": address}]}).sort('timestamp', 1)
        cc = list(convos)

        idz = conversations.insert(clean_conversations_and_add_poi(cc, poi=address))
        print "finished with {}".format(address)
        return address
    finally:
        conn.close()

In [ ]:
# from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor()

results = list(e.map(multi, addresses[100:150]))

In [25]:
for address in addresses[100:120]:
    conn, db, conversations, raw = mongo_connect()
    try:
        convos = raw.find({"$or": [{"from_addr": address}, {"to_addr": address}]}).sort('timestamp', 1)
        cc = list(convos)

        idz = conversations.insert(clean_conversations_and_add_poi(cc, poi=address))
        print "finished with {}".format(address)
    finally:
        conn.close()

finished with +27748024688
finished with +27764914783
finished with +27797247289
finished with +27797513653
finished with +27818660691
finished with +27761924314
finished with +27769544257
finished with +27767555006
finished with +27824721156
finished with +27799059646
finished with +27711754784
finished with +27766763040
finished with +27616388545
finished with +27792394957
finished with +27711328459


KeyboardInterrupt: 